<a href="https://colab.research.google.com/github/danielpatrickhug/community_weather_api/blob/main/community_weather_api_2_apis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import dataclasses
from dataclasses import dataclass
import datetime
import json

In [23]:
@dataclass 
class City:
    name: str
    weather_data: "CityWeatherSample"
    daily_local_news: "CityNews"


In [27]:
@dataclass
class CityNews:
    q: str
    sample_time: datetime.datetime
    top_n_articles: list

@dataclass
class ClimateNews:
    _type: str
    name: str
    url: str
    description: str
    about: list
    datePublished: str

    def __init__(self, **kwargs):
        names = set([f.name for f in dataclasses.fields(self)])
        for k, v in kwargs.items():
            if k in names:
                setattr(self, k, v)

    

In [28]:
@dataclass
class CityWeatherSample:
    q: str
    sample_time: datetime.datetime
    weather_desciption: "WeatherDescription"
    weather_main: "WeatherMain"
    coordinates: "Coordinates"

@dataclass
class WeatherDescription:
    id: int
    main: str
    description: str
    icon: str

@dataclass
class WeatherMain:
    temp: float
    feels_like: float
    temp_min: float
    temp_max: float
    pressure: float
    humidity: float


@dataclass
class Coordinates:
    lon: float
    lat: float

def build_weather_query_string(city_query):
    return {"q":f"{city_query}","units":"imperial","mode":"json"}

def build_bing_query_string(city_query):
    return {"q":f"{city_query} Climate News","freshness":"Day","textFormat":"Raw","safeSearch":"Off"}

In [ ]:
weather_api = "https://community-open-weather-map.p.rapidapi.com/weather"
weather_headers = {
	"X-RapidAPI-Host": "community-open-weather-map.p.rapidapi.com",
	"X-RapidAPI-Key": ""
}
bing_url = "https://bing-news-search1.p.rapidapi.com/news/search"
bing_headers = {
	"X-BingApis-SDK": "true",
	"X-RapidAPI-Host": "bing-news-search1.p.rapidapi.com",
	"X-RapidAPI-Key": ""
}


city_list = ["London,uk", "New York City,us", "Chicago,us", "Austin,us", "San Francisco,us", "Honolulu,us", "Tokyo,jp", "New Delhi,in", "Zurich"]
#ct = datetime.datetime.now()
for city in city_list:
    weather_query_string = build_weather_query_string(city)
    weather_response = requests.request("GET", weather_api, headers=weather_headers, params=weather_query_string)
    weather_data = weather_response.json()


    coords = Coordinates(**weather_data['coord'])
    desc = WeatherDescription(**weather_data['weather'][0])
    main = WeatherMain(**weather_data['main'])
    
    now = datetime.datetime.now()

    city_weather_sample = CityWeatherSample(q=city, sample_time=now, weather_desciption=desc, weather_main=main, coordinates=coords)

    bing_query_string = build_bing_query_string(city)
    bing_response = requests.request("GET", bing_url, headers=bing_headers, params=bing_query_string)
    bing_data = bing_response.json()
    #print(bing_data['value'])
    news = []
    for line in bing_data['value']:
        news.append(ClimateNews(**line))
    city_news = CityNews(q=f"{city} Climate News", sample_time=now, top_n_articles=news)


    city = City(name=city, weather_data=city_weather_sample, daily_local_news=city_news)
    print(city)